<a href="https://colab.research.google.com/github/saerarawas/AAI_635O_B11_202520-Recommender-System/blob/main/Week3/Saera_Rawas_Graph_based_Rec_sys_May_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing required libraries**

In [ ]:
!pip install py2neo

  Using cached py2neo-2021.2.4-py2.py3-none-any.whl.metadata (9.9 kB)
  Using cached interchange-2021.0.4-py2.py3-none-any.whl.metadata (1.9 kB)
  Using cached monotonic-1.6-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached pansi-2024.11.0-py2.py3-none-any.whl.metadata (3.1 kB)
Using cached py2neo-2021.2.4-py2.py3-none-any.whl (177 kB)
Using cached interchange-2021.0.4-py2.py3-none-any.whl (28 kB)
Using cached pansi-2024.11.0-py2.py3-none-any.whl (26 kB)
Using cached monotonic-1.6-py2.py3-none-any.whl (8.2 kB)


In [ ]:
!pip install openpyxl --upgrade

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.1.2
    Uninstalling openpyxl-3.1.2:
      Successfully uninstalled openpyxl-3.1.2


In [ ]:
!pip install neo4j

  Using cached neo4j-5.28.1-py3-none-any.whl.metadata (5.9 kB)
Using cached neo4j-5.28.1-py3-none-any.whl (312 kB)


In [ ]:
!pip install neo4jupyter

  Using cached neo4jupyter-0.2.0-py3-none-any.whl
  Using cached ipython_cypher-0.2.6-py3-none-any.whl
  Using cached neo4jrestclient-2.1.1-py2.py3-none-any.whl
  Using cached prettytable-3.16.0-py3-none-any.whl.metadata (33 kB)
Using cached prettytable-3.16.0-py3-none-any.whl (33 kB)


**Importing Required Libraries**

In [ ]:
import pandas as pd

from neo4j import GraphDatabase, basic_auth
from py2neo import Graph

import re

import neo4jupyter

**Establishing a connection between our Neo4j database and python notebook**

In [ ]:
g = Graph("bolt://localhost:7687", password = "1234567890")

In [ ]:
driver = GraphDatabase.driver(
  "bolt://localhost:7687",
  auth=basic_auth("neo4j", "1234567890"))

# The uri "bolt://34.201.241.51:7687" needs to be replaced in case of new sandbox creation in neo4j.
# The credentials "neo4j" and "whirls-bullet-boils" also need replacement for each use case.

In [ ]:
def execute_transactions(transaction_execution_commands):
    # Establishing connection with database
    data_base_connection = GraphDatabase.driver( "bolt://localhost:7687",
  auth=basic_auth("neo4j", "1234567890"))
    # Creating a session
    session = data_base_connection.session()
    for i in transaction_execution_commands:
        session.run(i)

**Loading the datasets**

In [ ]:
# This dataset consists of transactions which will be used to establish relationship between the customer and the stock.
df = pd.read_excel(r'Rec_sys_data.xlsx')

In [ ]:
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,DeliveryDate,Discount%,ShipMode,ShippingCost,CustomerID
0,536365,84029E,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.20,ExpressAir,30.12,17850
1,536365,71053,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.21,ExpressAir,30.12,17850
2,536365,21730,6,2010-12-01 08:26:00,2010-12-03 08:26:00,0.56,Regular Air,15.22,17850
3,536365,84406B,8,2010-12-01 08:26:00,2010-12-03 08:26:00,0.30,Regular Air,15.22,17850
4,536365,22752,2,2010-12-01 08:26:00,2010-12-04 08:26:00,0.57,Delivery Truck,5.81,17850


In [ ]:
# Little bit of preprocessing so that we can easily run NoSQL queries.
df['CustomerID'] = df['CustomerID'].apply(str)

In [ ]:
# This dataset contains detailed information about each stock which will be used to link stockcodes and their description/title.
df1 = pd.read_excel('Rec_sys_data.xlsx','product')

In [ ]:
df1.head()

,StockCode,Product Name,Description,Category,Brand,Unit Price
0,22629,Ganma Superheroes Ordinary Life Case For Samsu...,"New unique design, great gift.High quality pla...",Cell Phones|Cellphone Accessories|Cases & Prot...,Ganma,13.99
1,21238,Eye Buy Express Prescription Glasses Mens Wome...,Rounded rectangular cat-eye reading glasses. T...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,19.22
2,22181,MightySkins Skin Decal Wrap Compatible with Ni...,Each Nintendo 2DS kit is printed with super-hi...,Video Games|Video Game Accessories|Accessories...,Mightyskins,14.99
3,84879,Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...,The sheerest compression stocking in its class...,Health|Medicine Cabinet|Braces & Supports,Medi,62.38
4,84836,Stupell Industries Chevron Initial Wall D cor,Features: -Made in the USA. -Sawtooth hanger o...,Home Improvement|Paint|Wall Decals|All Wall De...,Stupell Industries,35.99


**Uploading the entities in Neo4J Database**

To implement knowledge graphs in Neo4J we need to convert the dataframe into Relational Database. For this, we need to convert customers and stocks into entities (or nodes of a graph) and build a relation between them.

In [ ]:
# creating a list of all unique customer IDs
customerids = df['CustomerID'].unique().tolist()

# storing all the create commands to be executed into create_customers list
create_customers = []

for i in customerids:
  # example of create statement "create (n:entity {property_key : '12345'})"
  statement = "create (c:customer{cid:"+ '"' + str(i) + '"' +"})"
  create_customers.append(statement)

# running all the queries into neo4j to create customer entities
execute_transactions(create_customers)

Once we are done with customer nodes, we need to create nodes for stocks as well.

In [ ]:
# creating a lit of all unique stockcodes
stockcodes = df['StockCode'].unique().tolist()

# storing all the create commands to be executed into the create_stockcodes list
create_stockcodes = []

for i in stockcodes:
  # example of create statement "create (m:entity {property_key : 'XYZ'})"
  statement = "create (s:stock{stockcode:"+ '"' + str(i) + '"' +"})"
  create_stockcodes.append(statement)

# running all the queries into neo4j to create stock entities
execute_transactions(create_stockcodes)

Once we are done creating nodes for customers and stocks, we need to create a link between stockcodes and title which will be needed to recommend items.


For this we will create another property key called as 'title' into the stock entity already present in our neo4j database.

In [ ]:
# creating a blank dataframe
df2 = pd.DataFrame(columns = ['StockCode', 'Title'])

In [ ]:
# This cell of code will add all the unique stockcodes along with their title in df2
stockcodes = df['StockCode'].unique().tolist()

for i in range(len(stockcodes)):
  df2['StockCode'][i] = stockcodes[i]
  df2['Title'][i] = df1[df1['StockCode']==stockcodes[i]]['Product Name'].values

C:\Users\Faisal\AppData\Local\Temp\ipykernel_19276\1517362260.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df2['StockCode'][i] = stockcodes[i]
C:\Users\Faisal\AppData\Local\Temp\ipykernel_19276\1517362260.py:6: FutureWarning: ChainedA

In [ ]:
# Doing some data preprocessing such that these queries can be run in neo4j
df2['Title'] = df2['Title'].apply(str)
df2['Title'] = df2['Title'].map(lambda x: re.sub(r'\W+', ' ', x))
df2['Title'] = df2['Title'].apply(str)

In [ ]:
# This query will add the 'title' property key to each stock entity in our neo4j database
for i in range(len(df2)):
  query = """
  MATCH (s:stock {stockcode:""" + '"' + str(df2['StockCode'][i]) + '"' + """})
  SET s.title ="""+ '"' + str(df2['Title'][i]) + '"' + """
  RETURN s.stockcode, s.title
  """

  g.run(query)

**Creating relation between customers and stocks**

Since we have all the transactions in our dataset, the relation is already known and present. But since we have to convert it into a RDS, we have to run cypher queries in neo4j to build this relationship.

In [ ]:
# Storing transaction values in a list
transaction_list = df.values.tolist()

# storing all commands to build relationship in an empty list relation
relation = []

for i in transaction_list:
  # the 9th column in df is customerID and 2nd column is stockcode which we are appending in the statement
  statement = """MATCH (a:customer),(b:stock) WHERE a.cid = """+'"' + str(i[8])+ '"' + """ AND b.stockcode = """ + '"' + str(i[1]) + '"' + """ CREATE (a)-[:bought]->(b) """
  relation.append(statement)

In [ ]:
execute_transactions(relation)

**Finding similarity between users using the relationship created**

In [ ]:
def similar_users(id) :
  # This query will find users who have bought stocks in common with the customer having id specified by user
  # Later we will find jaccard index for each of them
  # We wil return the neighbors sorted by jaccard index in descending order
  query = """
  MATCH (c1:customer)-[:bought]->(s:stock)<-[:bought]-(c2:customer)
  WHERE c1 <> c2 AND c1.cid =""" + '"' + str(id) +'"' """
  WITH c1, c2, COUNT(DISTINCT s) as intersection
  MATCH (c:customer)-[:bought]->(s:stock)
  WHERE c in [c1, c2]
  WITH c1, c2, intersection, COUNT(DISTINCT s) as union
  WITH c1, c2, intersection, union, (intersection * 1.0 / union) as jaccard_index
  ORDER BY jaccard_index DESC, c2.cid
  WITH c1, COLLECT([c2.cid, jaccard_index, intersection, union])[0..15] as neighbors
  WHERE SIZE(neighbors) = 15   // return users with enough neighbors
  RETURN c1.cid as customer, neighbors

  """
  neighbors = pd.DataFrame([['CustomerID','JaccardIndex','Intersection','Union']])
  for i in g.run(query).data():
    neighbors = neighbors.append(i["neighbors"])

  print("\n----------- customer's 15 nearest neighbors ---------\n")
  print(neighbors)


In [ ]:
def similar_users(id):
    query = """
    MATCH (c1:customer)-[:bought]->(s:stock)<-[:bought]-(c2:customer)
    WHERE c1 <> c2 AND c1.cid =""" + '"' + str(id) + '"' + """
    WITH c1, c2, COUNT(DISTINCT s) as intersection
    MATCH (c:customer)-[:bought]->(s:stock)
    WHERE c in [c1, c2]
    WITH c1, c2, intersection, COUNT(DISTINCT s) as union
    WITH c1, c2, intersection, union, (intersection * 1.0 / union) as jaccard_index
    ORDER BY jaccard_index DESC, c2.cid
    WITH c1, COLLECT([c2.cid, jaccard_index, intersection, union])[0..15] as neighbors
    WHERE SIZE(neighbors) = 15
    RETURN c1.cid as customer, neighbors
    """

    neighbor_rows = []
    for i in g.run(query).data():
        neighbor_rows.extend(i["neighbors"])

    neighbors_df = pd.DataFrame(neighbor_rows, columns=['CustomerID','JaccardIndex','Intersection','Union'])

    print("\n----------- customer's 15 nearest neighbors ---------\n")
    print(neighbors_df)


*Sample Outputs*

In [ ]:
similar_users('12347')


----------- customer's 15 nearest neighbors ---------

   CustomerID  JaccardIndex  Intersection  Union
0       17396      0.144444            13     90
1       13821      0.108333            13    120
2       17097      0.107784            18    167
3       13759      0.104167            10     96
4       13324      0.103093            10     97
5       15658      0.099099            11    111
6       15606      0.097345            11    113
7       16389      0.093750             9     96
8       18092      0.092784             9     97
9       13814      0.091743            10    109
10      13265      0.088710            11    124
11      13488      0.087248            26    298
12      12843      0.086957            12    138
13      16618      0.086207            10    116
14      15502      0.084444            19    225


In [ ]:
similar_users(17975)


----------- customer's 15 nearest neighbors ---------

   CustomerID  JaccardIndex  Intersection  Union
0       15356      0.127976            43    336
1       18231      0.126531            31    245
2       14395      0.125436            36    287
3       15856      0.124668            47    377
4       16907      0.124424            27    217
5       17787      0.123404            29    235
6       13344      0.122047            31    254
7       15059      0.112360            30    267
8       16222      0.111111            29    261
9       17085      0.108949            28    257
10      17865      0.106796            33    309
11      16910      0.106061            35    330
12      17450      0.106061            28    264
13      16549      0.105919            34    321
14      14410      0.102389            30    293


In [ ]:
similar_users(16359)


----------- customer's 15 nearest neighbors ---------

   CustomerID  JaccardIndex  Intersection  Union
0       15319      0.148936             7     47
1       15124      0.146341             6     41
2       13706      0.137255             7     51
3       15985      0.135593             8     59
4       14959      0.126984             8     63
5       13900      0.126761             9     71
6       16989      0.125000             5     40
7       14209      0.121951            10     82
8       16721      0.119403             8     67
9       15076      0.115385             9     78
10      15643      0.112903             7     62
11      14226      0.111111             9     81
12      15681      0.111111             6     54
13      14639      0.110169            13    118
14      12842      0.107692             7     65


In [ ]:
similar_users(17867)


----------- customer's 15 nearest neighbors ---------

   CustomerID  JaccardIndex  Intersection  Union
0       13551      0.200000             2     10
1       16566      0.133333             2     15
2       12917      0.111111             1      9
3       13967      0.111111             1      9
4       15442      0.111111             1      9
5       17483      0.105263             2     19
6       13217      0.100000             1     10
7       17384      0.100000             1     10
8       16526      0.095238             2     21
9       17854      0.095238             2     21
10      15160      0.090909             1     11
11      13207      0.083333             1     12
12      18032      0.083333             1     12
13      13805      0.076923             1     13
14      14145      0.076923             1     13


In [ ]:
similar_users(17850)


----------- customer's 15 nearest neighbors ---------

   CustomerID  JaccardIndex  Intersection  Union
0       15497      0.148148             4     27
1       17169      0.147059             5     34
2       18170      0.142857             6     42
3       15636      0.138889             5     36
4       13187      0.137255             7     51
5       15722      0.127660             6     47
6       15482      0.117647             4     34
7       13161      0.115385             3     26
8       14035      0.114286             4     35
9       17866      0.111111             3     27
10      13884      0.107143             6     56
11      14440      0.106383             5     47
12      14970      0.097561             4     41
13      17742      0.096774             3     31
14      14163      0.096154             5     52


**Recommending items based on the similar users**

In [ ]:
def recommend(id):
  # The query below is same as similar_users function
  # It will return the most similar customers
  query1 = """
  MATCH (c1:customer)-[:bought]->(s:stock)<-[:bought]-(c2:customer)
  WHERE c1 <> c2 AND c1.cid =""" + '"' + str(id) +'"' """
  WITH c1, c2, COUNT(DISTINCT s) as intersection
  MATCH (c:customer)-[:bought]->(s:stock)
  WHERE c in [c1, c2]
  WITH c1, c2, intersection, COUNT(DISTINCT s) as union
  WITH c1, c2, intersection, union, (intersection * 1.0 / union) as jaccard_index
  ORDER BY jaccard_index DESC, c2.cid
  WITH c1, COLLECT([c2.cid, jaccard_index, intersection, union])[0..15] as neighbors
  WHERE SIZE(neighbors) = 15   // return users with enough neighbors
  RETURN c1.cid as customer, neighbors

  """
  neighbors = pd.DataFrame([['CustomerID','JaccardIndex','Intersection','Union']])
  neighbors_list = {}
  for i in g.run(query1).data():
    neighbors = neighbors.append(i["neighbors"])
    neighbors_list[i["customer"]] = i["neighbors"]
    print(neighbors_list)

  # From the neighbors_list returned, we will fetch the customer ids of those neighbors to recommend items
  nearest_neighbors = [neighbors_list[id][i][0] for i in range(len(neighbors_list[id]))]

  # The below query will fetch all the items boughts by nearest neighbors
  # We will remove the items which have been already bought by the target customer
  # Now from the filtered set of items, we will count how many times each item is repeating within the shopping carts of nearest neighbors
  # We will sort that list on count of repititions and return in descending order
  query2 = """
        // get top n recommendations for customer from their nearest neighbors
        MATCH (c1:customer),(neighbor:customer)-[:bought]->(s:stock)    // all items bought by neighbors
        WHERE c1.cid = """ + '"' + str(id) + '"' """
          AND neighbor.cid in $nearest_neighbors
          AND not (c1)-[:bought]->(s)                    // filter for items that our user hasn't bought before

        WITH c1, s, COUNT(DISTINCT neighbor) as countnns // times bought by nearest neighbors
        ORDER BY c1.cid, countnns DESC
        RETURN c1.cid as customer, COLLECT([s.title, s.stockcode, countnns])[0..$n] as recommendations
        """

  recommendations = pd.DataFrame([['Title','StockCode','Number of times bought by neighbors']])
  for i in g.run(query2, id = id, nearest_neighbors = nearest_neighbors, n = 5).data():
    #recommendations[i["customer"]] = i["recommendations"]
    recommendations = recommendations.append(i["recommendations"])


  # We will also print the items bought earlier by the target customer
  print(" \n---------- Top 8 StockCodes bought by customer " + str(id) + " -----------\n")

  print(df[df['CustomerID']==id][['CustomerID','StockCode','Quantity']].nlargest(8,'Quantity'))

  bought = df[df['CustomerID']==id][['CustomerID','StockCode','Quantity']].nlargest(8,'Quantity')

  print('\n-------Product Name of bought StockCodes ------\n')

  print((df1[df1.StockCode.isin(bought.StockCode)]['Product Name']).to_string())

  # Here we will print the recommendations
  print("------------ \n Recommendations for Customer {} ------- \n".format(id))
  print(recommendations.to_string())


In [ ]:
def recommend(id):
    # Step 1: Get similar users
    query1 = """
    MATCH (c1:customer)-[:bought]->(s:stock)<-[:bought]-(c2:customer)
    WHERE c1 <> c2 AND c1.cid =""" + '"' + str(id) + '"' + """
    WITH c1, c2, COUNT(DISTINCT s) as intersection
    MATCH (c:customer)-[:bought]->(s:stock)
    WHERE c in [c1, c2]
    WITH c1, c2, intersection, COUNT(DISTINCT s) as union
    WITH c1, c2, intersection, union, (intersection * 1.0 / union) as jaccard_index
    ORDER BY jaccard_index DESC, c2.cid
    WITH c1, COLLECT([c2.cid, jaccard_index, intersection, union])[0..15] as neighbors
    WHERE SIZE(neighbors) = 15
    RETURN c1.cid as customer, neighbors
    """

    neighbors_list = {}
    neighbors_rows = []

    for i in g.run(query1).data():
        neighbors_rows.extend(i["neighbors"])
        neighbors_list[i["customer"]] = i["neighbors"]

    neighbors = pd.DataFrame(neighbors_rows, columns=['CustomerID', 'JaccardIndex', 'Intersection', 'Union'])

    # Step 2: Get neighbor IDs for recommendation
    nearest_neighbors = [neighbors_list[id][i][0] for i in range(len(neighbors_list[id]))]

    # Step 3: Get item recommendations
    query2 = """
    MATCH (c1:customer),(neighbor:customer)-[:bought]->(s:stock)
    WHERE c1.cid = """ + '"' + str(id) + '"' + """
      AND neighbor.cid in $nearest_neighbors
      AND NOT (c1)-[:bought]->(s)
    WITH c1, s, COUNT(DISTINCT neighbor) as countnns
    ORDER BY c1.cid, countnns DESC
    RETURN c1.cid as customer, COLLECT([s.title, s.stockcode, countnns])[0..$n] as recommendations
    """

    recommendation_rows = []
    for i in g.run(query2, id=id, nearest_neighbors=nearest_neighbors, n=5).data():
        recommendation_rows.extend(i["recommendations"])

    recommendations = pd.DataFrame(recommendation_rows, columns=['Title', 'StockCode', 'Number of times bought by neighbors'])

    # Step 4: Show previously bought items
    print(f"\n---------- Top 8 StockCodes bought by customer {id} -----------\n")
    bought = df[df['CustomerID'] == id][['CustomerID', 'StockCode', 'Quantity']].nlargest(8, 'Quantity')
    print(bought)

    print('\n------- Product Names of bought StockCodes ------\n')
    print(df1[df1.StockCode.isin(bought.StockCode)]['Product Name'].to_string())

    # Step 5: Show recommendations
    print(f"\n------------ Recommendations for Customer {id} -----------\n")
    print


In [ ]:
recommend('17850')


---------- Top 8 StockCodes bought by customer 17850 -----------

     CustomerID StockCode  Quantity
285       17850    82494L        12
2629      17850    85123A        12
2634      17850     71053        12
2978      17850     71053        12
2983      17850    85123A        12
3299      17850     71053        12
3301      17850     21068        12
3302      17850    84029G        12

------- Product Names of bought StockCodes ------

135    Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...
162    Heavy Duty Handlebar Motorcycle Mount Holder K...
179           AARCO Enclosed Wall Mounted Bulletin Board
669    3 1/2"W x 20"D x 20"H Funston Craftsman Smooth...
967    Awkward Styles Shamrock Flag St. Patrick's Day...

------------ Recommendations for Customer 17850 -----------



In [ ]:
recommend("16359")


---------- Top 8 StockCodes bought by customer 16359 -----------

       CustomerID StockCode  Quantity
161089      16359     84836       120
232132      16359     84836        72
101750      16359     84836        60
124252      16359     84836        60
53524       16359    16169M        50
75394       16359     84836        48
33114       16359     84836        36
53526       16359     84836        36

------- Product Names of bought StockCodes ------

4         Stupell Industries Chevron Initial Wall D cor
3046    Harriton Ladies' 4.2 oz. Athletic Sport T-Shirt

------------ Recommendations for Customer 16359 -----------



In [ ]:
recommend("17867")


---------- Top 8 StockCodes bought by customer 17867 -----------

       CustomerID StockCode  Quantity
137223      17867    18097C       288
188192      17867     23109        42
188161      17867     85103        37
64167       17867     21633         2
188162      17867    15056N         2
137222      17867     21631         1
137224      17867    72351B         1
137225      17867    47599A         1

------- Product Names of bought StockCodes ------

89                Window Tint Film Chevy (back doors) DIY
261     PeanutsÃƒâ€šÃ‚ Valentine Sign Craft Kit (Pack ...
371     Urbane Urbane Classic Crossover Scrub Top Scru...
902                "Soccer" Vinyl Graphic - Large - Ivory
1591    Handcrafted Ercolano Music Box Featuring "Lunc...
2513    Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...
2739    Edwards Garment Women's Two Pockets Chino Blen...
3069    MightySkins Skin Decal Wrap Compatible with HP...

------------ Recommendations for Customer 17867 -----------



In [ ]:
recommend("12347")


---------- Top 8 StockCodes bought by customer 12347 -----------

       CustomerID StockCode  Quantity
99443       12347     23076       240
10526       12347     22492        36
99444       12347     22492        36
153949      12347     17021        36
200488      12347    84558A        36
10527       12347    85167B        30
10534       12347    84558A        24
43446       12347     84991        24

------- Product Names of bought StockCodes ------

33           Rosalind Wheeler Wall Mounted Bulletin Board
447     Eye Buy Express Kids Childrens Reading Glasses...
782     6pc Boy Formal Necktie Black & White Suit Set ...
1607    3 1/2"W x 32"D x 36"H Traditional Arts & Craft...
1820        Fruit of the Loom T-Shirts HD Cotton Tank Top
2668    Vickerman 14" Finial Drop Christmas Ornaments,...

------------ Recommendations for Customer 12347 -----------

